<a href="https://colab.research.google.com/github/IgorB99/Pokemon_Battle/blob/master/Pokemon_Colab_Color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
class Pokemon():
  def __init__(self, name, hp, attack, defense, special_attack, special_defense, speed):
    self.name = name
    self.hp = hp
    self.attack = attack
    self.defense = defense
    self.special_attack = special_attack
    self.special_defense = special_defense
    self.speed = speed

  def show_data(self):
    print('name:{}, hp: {}, attack:{}, defense:{}, special_attack:{}, special_defense:{}, speed:{}'.format( \
    self.name, self.hp, self.attack, self.defense, self.special_attack, self.special_defense, self.speed))

In [44]:
import requests

base_api_url = 'https://pokeapi.co/api/v2/pokemon?limit=151'
dados_brutos = requests.get(base_api_url)

def quem_e_esse_pokemon(pokemon):
  if dados_brutos.status_code >= 200 and dados_brutos.status_code < 300:
    for informacao in dados_brutos.json()["results"]:
      if informacao["name"] == pokemon:
        dados_brutos_pokemon = requests.get(informacao["url"])
        if dados_brutos_pokemon.status_code >= 200 and dados_brutos_pokemon.status_code < 300:
          pokemon_data = dados_brutos_pokemon.json()
          pokemon = Pokemon(name = pokemon_data["name"],
                            hp = pokemon_data["stats"][0]["base_stat"],
                            attack = pokemon_data["stats"][1]["base_stat"],
                            defense = pokemon_data["stats"][2]["base_stat"],
                            special_attack = pokemon_data["stats"][3]["base_stat"],
                            special_defense = pokemon_data["stats"][4]["base_stat"],
                            speed = pokemon_data["stats"][5]["base_stat"])
  return pokemon

pokemon1 = input("digite o nome de um pokemon: ")
pokemon2 = input("digite o nome de um pokemon: ")

pokemon1 = quem_e_esse_pokemon(pokemon1)
pokemon2 = quem_e_esse_pokemon(pokemon2)

while pokemon1.hp > 0 or pokemon2.hp > 0:
  pokemon1_action = input(pokemon1.name + " use (attack/special_attack): ")
  pokemon2_action = input(pokemon2.name + " use (attack/special_attack): ")

  if pokemon1.speed > pokemon2.speed:
    pokemon2.hp -= pokemon1.pokemon1_action
  else:
    pokemon1.hp -= pokemon2.pokemon2_action

if pokemon1.hp > pokemon2.hp:
  print(pokemon1.name + " win")
else:
  print(pokemon2.name + " win")

digite o nome de um pokemon: mew
digite o nome de um pokemon: mewtwo
mew use (attack/special_attack): attack
mewtwo use (attack/special_attack): attack


AttributeError: ignored